In [1]:
import pandas as pd

df7 = pd.read_csv('test.csv', encoding='UTF-8', sep=',')

print(df7)

            ID Customer_ID      Month             Name   Age          SSN  \
0       0x160a   CUS_0xd40  September    Aaron Maashoh    23  821-00-0265   
1       0x160b   CUS_0xd40    October    Aaron Maashoh    24  821-00-0265   
2       0x160c   CUS_0xd40   November    Aaron Maashoh    24  821-00-0265   
3       0x160d   CUS_0xd40   December    Aaron Maashoh   24_  821-00-0265   
4       0x1616  CUS_0x21b1  September  Rick Rothackerj    28  004-07-5839   
...        ...         ...        ...              ...   ...          ...   
49995  0x25fe5  CUS_0x8600   December   Sarah McBridec  4975  031-35-0942   
49996  0x25fee  CUS_0x942c  September            Nicks    25  078-73-5990   
49997  0x25fef  CUS_0x942c    October            Nicks    25  078-73-5990   
49998  0x25ff0  CUS_0x942c   November            Nicks    25  078-73-5990   
49999  0x25ff1  CUS_0x942c   December            Nicks    25  078-73-5990   

      Occupation Annual_Income  Monthly_Inhand_Salary  Num_Bank_Accounts  .

In [2]:
#Preprocesamiento y limpieza inicial dataset test

#Estadísticas descriptivas iniciales
print(df7.describe())

#Ver tipos de datos
print('Tipos de datos:')
print(df7.dtypes)

       Monthly_Inhand_Salary  Num_Bank_Accounts  Num_Credit_Card  \
count           42502.000000       50000.000000     50000.000000   
mean             4182.004291          16.838260        22.921480   
std              3174.109304         116.396848       129.314804   
min               303.645417          -1.000000         0.000000   
25%              1625.188333           3.000000         4.000000   
50%              3086.305000           6.000000         5.000000   
75%              5934.189094           7.000000         7.000000   
max             15204.633333        1798.000000      1499.000000   

       Interest_Rate  Delay_from_due_date  Num_Credit_Inquiries  \
count   50000.000000         50000.000000          48965.000000   
mean       68.772640            21.052640             30.080200   
std       451.602363            14.860397            196.984121   
min         1.000000            -5.000000              0.000000   
25%         8.000000            10.000000           

In [3]:
#Identificamos caracteres extraños en cada columna

import re

# Función para identificar caracteres extraños en cada columna
def detectar_caracteres_extranos(df7):
    caracteres_por_columna = {}
    for col in df7.columns:
        valores_unicos = ''.join(df7[col].astype(str).unique())  # Unir todos los valores únicos como string
        caracteres_unicos = set(re.findall(r'[^a-zA-Z0-9\s.,-]', valores_unicos))  # Buscar caracteres no alfanuméricos comunes
        if caracteres_unicos:  # Solo guardar columnas con caracteres extraños
            caracteres_por_columna[col] = caracteres_unicos
    return caracteres_por_columna

# Identificar caracteres extraños en el dataframe
caracteres_extranos = detectar_caracteres_extranos(df7)

# Mostrar los resultados
for columna, caracteres in caracteres_extranos.items():
    print(f"Columna: {columna} -> Caracteres extraños detectados: {caracteres}")

Columna: Customer_ID -> Caracteres extraños detectados: {'_'}
Columna: Name -> Caracteres extraños detectados: {'"'}
Columna: Age -> Caracteres extraños detectados: {'_'}
Columna: SSN -> Caracteres extraños detectados: {'@', '%', '#', '$', '&', '*'}
Columna: Occupation -> Caracteres extraños detectados: {'_'}
Columna: Annual_Income -> Caracteres extraños detectados: {'_'}
Columna: Num_of_Loan -> Caracteres extraños detectados: {'_'}
Columna: Num_of_Delayed_Payment -> Caracteres extraños detectados: {'_'}
Columna: Changed_Credit_Limit -> Caracteres extraños detectados: {'_'}
Columna: Credit_Mix -> Caracteres extraños detectados: {'_'}
Columna: Outstanding_Debt -> Caracteres extraños detectados: {'_'}
Columna: Amount_invested_monthly -> Caracteres extraños detectados: {'_'}
Columna: Payment_Behaviour -> Caracteres extraños detectados: {'!', '_', '@', '%', '#'}
Columna: Monthly_Balance -> Caracteres extraños detectados: {'_'}


In [4]:
import numpy as np

#Aplicar limpieza caracteres extraños
#Función para limpiar el dataset según las reglas definidas
def limpiar_columnas(df7):
    #Eliminar guiones bajos y convertir a numérico en columnas específicas
    df7['Age'] = df7['Age'].str.replace('_','', regex=True).astype(float)
    df7['Annual_Income'] = df7['Annual_Income'].str.replace('_','',regex=True).astype(float)

    #Reemplazar guiones bajos por espacios en columnas específicas
    columnas_reemplazo_espacio = ['Num_of_Loan','Num_of_Delayed_Payment','Changed_Credit_Limit','Credit_Mix',]
    for col in columnas_reemplazo_espacio:
        df7[col] = df7[col].str.replace('_',' ', regex=True)

    #Eliminar registros con caracteres inválidos
    df7 = df7[~df7['Payment_Behaviour'].str.contains(r'[!%@#]', regex=True)]

    #Reemplazar guines bajos por espacios en Payment_Behaviour
    df7['Payment_Behaviour'] = df7['Payment_Behaviour'].str.replace('_',' ', regex=True)

    return df7

#Aplicar limpieza al dataframe
df7_clean = limpiar_columnas(df7)

#Hacer conversiones adicionales
#Lista de columnas que deben ser numéricas
columnas_a_convertir = ['Num_of_Loan','Num_of_Delayed_Payment','Changed_Credit_Limit']

#Reemplazar espacios vacíos por NaN
df7_clean[columnas_a_convertir] = df7_clean[columnas_a_convertir].replace(' ', np.nan)

#Convertir a numérico
df7_clean[columnas_a_convertir] = df7_clean[columnas_a_convertir].astype(float)

#Verificar si aún hay NaN
print('Verificar si aún hay Nan en columnas a convertir')
print(df7_clean[columnas_a_convertir].isnull().sum())

#Revisar si hay columnas que deberían ser numéricas
num_cols = ['Monthly_Inhand_Salary', 'Num_Bank_Accounts', 'Outstanding_Debt', 'Num_of_Loan',
            'Num_of_Delayed_Payment','Changed_Credit_Limit',
            'Credit_Utilization_Ratio','Credit_History_Age','Total_EMI_per_month',
            'Amount_invested_monthly','Monthly_Balance']
for col in num_cols:
    df7[col] = pd.to_numeric(df7[col], errors='coerce')

#Verificar cambios
print(df7.dtypes)

C:\Users\esteb\AppData\Local\Temp\ipykernel_16804\3263409322.py:19: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df7['Payment_Behaviour'] = df7['Payment_Behaviour'].str.replace('_',' ', regex=True)
C:\Users\esteb\AppData\Local\Temp\ipykernel_16804\3263409322.py:31: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df7_clean[columnas_a_convertir] = df7_clean[columnas_a_convertir].replace(' ', np.nan)
C:\Users\esteb\AppData\Local\Temp\ipykernel_16804\3263409322.py:34: SettingWithCopyWarning: 
A value is trying t

Verificar si aún hay Nan en columnas a convertir
Num_of_Loan                  0
Num_of_Delayed_Payment    3212
Changed_Credit_Limit       977
dtype: int64
ID                           object
Customer_ID                  object
Month                        object
Name                         object
Age                         float64
SSN                          object
Occupation                   object
Annual_Income               float64
Monthly_Inhand_Salary       float64
Num_Bank_Accounts             int64
Num_Credit_Card               int64
Interest_Rate                 int64
Num_of_Loan                   int64
Type_of_Loan                 object
Delay_from_due_date           int64
Num_of_Delayed_Payment      float64
Changed_Credit_Limit        float64
Num_Credit_Inquiries        float64
Credit_Mix                   object
Outstanding_Debt            float64
Credit_Utilization_Ratio    float64
Credit_History_Age          float64
Payment_of_Min_Amount        object
Total_EMI_per_mon

In [5]:
#Diagnóstico de valores nulos
for i in df7_clean.columns:
    print(i+':'+ str(df7_clean[i].isnull().sum()))

print(df7_clean)

ID:0
Customer_ID:0
Month:0
Name:4641
Age:0
SSN:0
Occupation:0
Annual_Income:0
Monthly_Inhand_Salary:6949
Num_Bank_Accounts:0
Num_Credit_Card:0
Interest_Rate:0
Num_of_Loan:0
Type_of_Loan:5303
Delay_from_due_date:0
Num_of_Delayed_Payment:3212
Changed_Credit_Limit:977
Num_Credit_Inquiries:953
Credit_Mix:0
Outstanding_Debt:0
Credit_Utilization_Ratio:0
Credit_History_Age:4103
Payment_of_Min_Amount:0
Total_EMI_per_month:0
Amount_invested_monthly:2117
Payment_Behaviour:0
Monthly_Balance:522
            ID Customer_ID      Month             Name     Age          SSN  \
0       0x160a   CUS_0xd40  September    Aaron Maashoh    23.0  821-00-0265   
1       0x160b   CUS_0xd40    October    Aaron Maashoh    24.0  821-00-0265   
2       0x160c   CUS_0xd40   November    Aaron Maashoh    24.0  821-00-0265   
3       0x160d   CUS_0xd40   December    Aaron Maashoh    24.0  821-00-0265   
4       0x1616  CUS_0x21b1  September  Rick Rothackerj    28.0  004-07-5839   
...        ...         ...        ...

In [6]:
# Para manejar valores nulos, nos apoyaremos en el registro duplicado correspondiente (considerando que son datos de panel)

# Función de imputación por duplicados
def imputar_con_logica_duplicados(df7_clean, columnas_a_imputar):
    df7_clean = df7_clean.copy()

    #Ordenamos por customer_ID y Month para mantener coherencia temporal
    df7_clean = df7_clean.sort_values(by=['Customer_ID','Month']).reset_index(drop=True)

    for col in columnas_a_imputar:
        #Trabajamos por grupos de clientes
        def imputar_grupo(grupo):
            #Si todo está NaN en esa columna para ese cliente, lo dejamos tal cual
            if grupo[col].isnull().all():
                return grupo[col]
            
            #Aplicamos imputación con duplicados
            valores = grupo[col].copy()

            for idx in range(len(valores)):
                if pd.isna(valores.iloc[idx]):
                    if idx == 0: #Primer registro del cliente
                        siguiente_valido = valores.iloc[idx+1:].dropna()
                        if not siguiente_valido.empty:
                            valores.iloc[idx] = siguiente_valido.iloc[0]
                    elif idx == len(valores) - 1: #Ultimo registro
                        anterior_valido = valores.iloc[:idx].dropna()
                        if not anterior_valido.empty:
                            valores.iloc[idx] = anterior_valido.iloc[-1]
                    else: #Registro intermedio
                        siguiente_valido = valores.iloc[idx+1:].dropna()
                        anterior_valido = valores.iloc[:idx].dropna()
                        if not siguiente_valido.empty:
                            valores.iloc[idx] = siguiente_valido.iloc[0]
                        elif not anterior_valido.empty:
                            valores.iloc[idx] = anterior_valido.iloc[-1]
            return valores
        
        #Aplicamos la función a cada grupo de clientes
        df7_clean[col] = df7_clean.groupby('Customer_ID').apply(imputar_grupo).reset_index(level=0, drop=True)

    return df7_clean

# Aplicamos limpiza invocando la función a las columnas con valores NaN (del diagnóstico)
columnas_con_nan = [
    'Name','Monthly_Inhand_Salary', 'Type_of_Loan', 'Num_of_Delayed_Payment',
    'Changed_Credit_Limit', 'Num_Credit_Inquiries', 'Credit_History_Age',
    'Amount_invested_monthly', 'Monthly_Balance'
]

# Aplicar la función
df7_clean = imputar_con_logica_duplicados(df7_clean, columnas_con_nan)
   
print(df7_clean)

# Diagnosticamos nulos nuevamente
for i in df7_clean.columns:
    print(i+':'+ str(df7_clean[i].isnull().sum()))

print('\n Nuevamente al aplicar el mismo método de imputación y limpieza a este dataframe de testeo bajo la lógica de duplicados, se puede evidenciar ' \
'en el diagnóstico posterior que aún quedan valores nulos, esta vez presentes en más columnas que nuestro dataframe de entrenamiento. Es por ello, que se ' \
'utilizará el mismo método de imputación usando la moda del cliente, además de hacer los ajustes correspondientes.')

C:\Users\esteb\AppData\Local\Temp\ipykernel_16804\2854179094.py:40: DeprecationWarning: DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the grouping columns will be excluded from the operation. Either pass `include_groups=False` to exclude the groupings or explicitly select the grouping columns after groupby to silence this warning.
  df7_clean[col] = df7_clean.groupby('Customer_ID').apply(imputar_grupo).reset_index(level=0, drop=True)
C:\Users\esteb\AppData\Local\Temp\ipykernel_16804\2854179094.py:40: DeprecationWarning: DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the grouping columns will be excluded from the operation. Either pass `include_groups=False` to exclude the groupings or explicitly select the grouping columns after groupby to silence this warning.
  df7_clean[col] = df7_clean.groupby('Customer_ID').apply(imputar_grupo).reset_in

            ID Customer_ID      Month            Name   Age          SSN  \
0      0x16295  CUS_0x1000   December  Alistair Barrf  18.0  913-74-1218   
1      0x16294  CUS_0x1000   November  Alistair Barrf  18.0  913-74-1218   
2      0x16293  CUS_0x1000    October  Alistair Barrf  18.0  913-74-1218   
3      0x16292  CUS_0x1000  September  Alistair Barrf  18.0  913-74-1218   
4       0x66ad  CUS_0x1009   December          Arunah  26.0  063-67-6938   
...        ...         ...        ...             ...   ...          ...   
46195   0x61ee   CUS_0xffc  September           Brads  18.0  226-86-7294   
46196  0x25b05   CUS_0xffd   December        Damouniq  30.0  832-88-8320   
46197  0x25b04   CUS_0xffd   November        Damouniq  30.0    #F%$D@*&8   
46198  0x25b03   CUS_0xffd    October        Damouniq  30.0  832-88-8320   
46199  0x25b02   CUS_0xffd  September        Damouniq  30.0  832-88-8320   

      Occupation  Annual_Income  Monthly_Inhand_Salary  Num_Bank_Accounts  \
0         

C:\Users\esteb\AppData\Local\Temp\ipykernel_16804\2854179094.py:40: DeprecationWarning: DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the grouping columns will be excluded from the operation. Either pass `include_groups=False` to exclude the groupings or explicitly select the grouping columns after groupby to silence this warning.
  df7_clean[col] = df7_clean.groupby('Customer_ID').apply(imputar_grupo).reset_index(level=0, drop=True)


In [7]:
# Convertir la columna Credit_History_Age a años (de object a float)
def convertir_credit_history_a_anios(texto):
    import numpy as np
    if pd.isnull(texto):
        return np.nan
    try:
        partes = texto.lower().replace("and", "").replace("months","month").replace("years","year").split()
        anios = int(partes[0]) if 'year' in partes[1] else 0
        meses = int(partes[2]) if len(partes) > 2 and 'month' in partes[3] else 0
        return round(anios + meses /12, 2)
    except:
        return np.nan
    
df7_clean['Credit_History_Age'] = df7_clean['Credit_History_Age'].apply(convertir_credit_history_a_anios)

#Identificamos caracteres extraños en cada columna luego de la imputación inicial

import re

# Función para identificar caracteres extraños en cada columna
def detectar_caracteres_extranos(df7):
    caracteres_por_columna = {}
    for col in df7.columns:
        valores_unicos = ''.join(df7[col].astype(str).unique())  # Unir todos los valores únicos como string
        caracteres_unicos = set(re.findall(r'[^a-zA-Z0-9\s.,-]', valores_unicos))  # Buscar caracteres no alfanuméricos comunes
        if caracteres_unicos:  # Solo guardar columnas con caracteres extraños
            caracteres_por_columna[col] = caracteres_unicos
    return caracteres_por_columna

# Identificar caracteres extraños en el dataframe
caracteres_extranos = detectar_caracteres_extranos(df7)

# Mostrar los resultados
for columna, caracteres in caracteres_extranos.items():
    print(f"Columna: {columna} -> Caracteres extraños detectados: {caracteres}")

print('\n Nuevamente surge la problemática de los valores incoherentes y Nulos, se aplicarán los mismos métodos del set de entrenamiento (moda mediante ' \
'duplicados).')

Columna: Customer_ID -> Caracteres extraños detectados: {'_'}
Columna: Name -> Caracteres extraños detectados: {'"'}
Columna: SSN -> Caracteres extraños detectados: {'@', '%', '#', '$', '&', '*'}
Columna: Occupation -> Caracteres extraños detectados: {'_'}
Columna: Payment_Behaviour -> Caracteres extraños detectados: {'!', '_', '@', '%', '#'}

 Nuevamente surge la problemática de los valores incoherentes y Nulos, se aplicarán los mismos métodos del set de entrenamiento (moda mediante duplicados).


In [8]:
# Función de limpieza utilizando duplicados mediante Name

import pandas as pd
import numpy as np
import re

def limpieza_avanzada_por_duplicados(df, columna_name='Name'):
    df = df.copy()

    # Convertimos a numérico las columnas posibles para detección correcta
    for col in df.select_dtypes(include='object'):
        try:
            df[col] = pd.to_numeric(df[col], errors='ignore')
        except:
            pass

    # 1. Trabajar primero con columnas numéricas
    columnas_numericas = df.select_dtypes(include=['int64', 'float64']).columns

    for col in columnas_numericas:
        if col == columna_name:
            continue  # No modificar el nombre

        # Calcular la moda por el nombre del cliente
        moda_por_cliente = df.groupby(columna_name)[col].agg(
            lambda x: x.mode().iloc[0] if not x.mode().empty else np.nan
        )

        # Imputar donde hay NaN o donde el valor difiere de la moda
        for idx, row in df.iterrows():
            customer = row[columna_name]
            valor = row[col]
            moda = moda_por_cliente.get(customer, np.nan)

            if pd.isna(valor) or (not pd.isna(moda) and valor != moda):
                df.at[idx, col] = moda

        print(f"[DEBUG] Imputación numérica aplicada en '{col}'")

    # 2. Ahora columnas categóricas con caracteres extraños o NaN
    columnas_categoricas = df.select_dtypes(include='object').columns

    patron_extrano = re.compile(r'[^a-zA-Z0-9\s.,-]')

    for col in columnas_categoricas:
        if col == columna_name:
            continue

        def es_valor_extrano(x):
            if pd.isna(x): return True
            return bool(patron_extrano.search(str(x)))

        mascara_extranos = df[col].apply(es_valor_extrano)

        # Obtener moda categórica por nombre del cliente (para valores válidos)
        moda_cat_por_cliente = (
            df[~mascara_extranos].groupby(columna_name)[col]
            .agg(lambda x: x.mode().iloc[0] if not x.mode().empty else np.nan)
        )

        # Aplicar la imputación
        for idx in df[mascara_extranos].index:
            customer = df.at[idx, columna_name]
            moda = moda_cat_por_cliente.get(customer, np.nan)
            if pd.notna(moda):
                df.at[idx, col] = moda

        print(f"[DEBUG] Imputación categórica aplicada en '{col}'")

    return df

#Ejecutar función
df7_clean = limpieza_avanzada_por_duplicados(df7_clean)

print('\n Al ejecutar la función de limpieza para el dataframe de testeo, aparecen nuevos outliers en columnas conocidas: valores negativos extremos ' \
'que venían siendo la moda y ensucian los modelos de Clasificación. Es necesario realizar una limpieza adicional a estos valores numéricos para este dataframe.')

C:\Users\esteb\AppData\Local\Temp\ipykernel_16804\973997194.py:13: FutureWarning: errors='ignore' is deprecated and will raise in a future version. Use to_numeric without passing `errors` and catch exceptions explicitly instead
  df[col] = pd.to_numeric(df[col], errors='ignore')


[DEBUG] Imputación numérica aplicada en 'Age'
[DEBUG] Imputación numérica aplicada en 'Annual_Income'
[DEBUG] Imputación numérica aplicada en 'Monthly_Inhand_Salary'
[DEBUG] Imputación numérica aplicada en 'Num_Bank_Accounts'
[DEBUG] Imputación numérica aplicada en 'Num_Credit_Card'
[DEBUG] Imputación numérica aplicada en 'Interest_Rate'
[DEBUG] Imputación numérica aplicada en 'Num_of_Loan'
[DEBUG] Imputación numérica aplicada en 'Delay_from_due_date'
[DEBUG] Imputación numérica aplicada en 'Num_of_Delayed_Payment'
[DEBUG] Imputación numérica aplicada en 'Changed_Credit_Limit'
[DEBUG] Imputación numérica aplicada en 'Num_Credit_Inquiries'
[DEBUG] Imputación numérica aplicada en 'Credit_Utilization_Ratio'
[DEBUG] Imputación numérica aplicada en 'Credit_History_Age'
[DEBUG] Imputación numérica aplicada en 'Total_EMI_per_month'
[DEBUG] Imputación categórica aplicada en 'ID'
[DEBUG] Imputación categórica aplicada en 'Customer_ID'
[DEBUG] Imputación categórica aplicada en 'Month'
[DEBUG] Im

In [9]:
# Diagnóstico outliers incoherentes

import numpy as np

# Criterios de validez por columna (puedes ajustar los que estimes)
criterios_validacion = {
    'Age': lambda x: x > 0 and x < 122,
    'Annual_Income': lambda x: x > 0,
    'Monthly_Inhand_Salary': lambda x: x >= 0,
    'Num_Bank_Accounts': lambda x: x >= 0,
    'Num_Credit_Card': lambda x: x >= 0,
    'Interest_Rate': lambda x: x >= 0,
    'Num_of_Loan': lambda x: x >= 0,
    'Delay_from_due_date': lambda x: x >= 0,
    'Num_of_Delayed_Payment': lambda x: x >= 0,
    'Changed_Credit_Limit': lambda x: x >= 0,
    'Num_Credit_Inquiries': lambda x: x >= 0,
    'Outstanding_Debt': lambda x: x >= 0,
    'Credit_Utilization_Ratio': lambda x: x >= 0,
    'Credit_History_Age': lambda x: x >= 0,
    'Total_EMI_per_month': lambda x: x >= 0,
    'Amount_invested_monthly': lambda x: x >= 0,
    'Monthly_Balance': lambda x: x >= 0
}

# Creamos una copia para diagnóstico
df_test_diagnostico = df7_clean.copy()

# Forzamos conversión a numérico en las columnas con criterios
for col in criterios_validacion.keys():
    df_test_diagnostico[col] = pd.to_numeric(df_test_diagnostico[col], errors='coerce')

# Aplicamos los criterios y marcamos columnas válidas
for col, criterio in criterios_validacion.items():
    df_test_diagnostico[f'{col}_valido'] = df_test_diagnostico[col].apply(
        lambda x: criterio(x) if not pd.isna(x) else False
    )

# Verificamos cuántas filas tienen al menos un valor no válido
valid_cols = [f'{col}_valido' for col in criterios_validacion]
df_test_diagnostico['Fila_Valida'] = df_test_diagnostico[valid_cols].all(axis=1)

# Mostramos las filas no válidas
registros_invalidos = df_test_diagnostico[~df_test_diagnostico['Fila_Valida']]
print(f"Cantidad de registros con al menos un valor numérico inválido: {len(registros_invalidos)}")
print(registros_invalidos[[col for col in df_test_diagnostico.columns if '_valido' in col or col == 'Name']])

print('\n Al correr este diagnóstico adicional, nos podemos dar cuenta de que existen 1049 registros con valores numéricos inválidos. Sin embargo, aquí ' \
'hay que comunicar un hallazgo importante del proyecto: ambos dataframes (entrenamiento y testeo) contiene a los mismo clientes en distintos momentos del ' \
'tiempo (con periodicidad mensual) y se busca clasificar nuevamente el credit scoring de ellos en base a la evolución de sus descriptores crediticios. ' \
'Es por ello que esta vez, para imputar información nula faltante, se usará el dataframe de entrenamiento "df6_clean" (no el codificado) mediante Customer_ID para ' \
'cruzar la información de los clientes y arreglar estos valores numéricos incoherentes.')

Cantidad de registros con al menos un valor numérico inválido: 1049
                   Name  Age_valido  Annual_Income_valido  \
158             Gillesw        True                  True   
159             Gillesw        True                  True   
160             Gillesw        True                  True   
161             Gillesw        True                  True   
396    Mark Felsenthalk        True                  True   
...                 ...         ...                   ...   
46034      Tarmo Virkip        True                  True   
46035      Tarmo Virkip        True                  True   
46040             Huntt        True                  True   
46041             Huntt        True                  True   
46042             Huntt        True                  True   

       Monthly_Inhand_Salary_valido  Num_Bank_Accounts_valido  \
158                            True                      True   
159                            True                      True   
160 

In [10]:
# Importación de dataframe de entremiento limpio (sin codificar)
import pandas as pd

df6_clean = pd.read_csv('train_clean(sin_codificar).csv', encoding='UTF-8', sep=',')

print(df6_clean)

            ID Customer_ID     Month            Name   Age          SSN  \
0      0x16291  CUS_0x1000    August  Alistair Barrf  45.0  913-74-1218   
1      0x1628b  CUS_0x1000  February  Alistair Barrf  45.0  913-74-1218   
2      0x1628a  CUS_0x1000   January  Alistair Barrf  45.0  913-74-1218   
3      0x16290  CUS_0x1000      July  Alistair Barrf  45.0  913-74-1218   
4      0x1628f  CUS_0x1000      June  Alistair Barrf  45.0  913-74-1218   
...        ...         ...       ...             ...   ...          ...   
92395  0x25afa   CUS_0xffd   January        Damouniq  29.0  832-88-8320   
92396  0x25b00   CUS_0xffd      July        Damouniq  29.0  832-88-8320   
92397  0x25aff   CUS_0xffd      June        Damouniq  29.0  832-88-8320   
92398  0x25afc   CUS_0xffd     March        Damouniq  29.0  832-88-8320   
92399  0x25afe   CUS_0xffd       May        Damouniq  29.0  832-88-8320   

      Occupation  Annual_Income  Monthly_Inhand_Salary  Num_Bank_Accounts  \
0         Lawyer      

In [11]:
# Imputamos valores inválidos de df7_clean usando df6_clean

#Hacemos una copia para imputar sobre ella
df7_imputado = df7_clean.copy()

#Nos aseguramos de que ambas columnas de Customer_ID estén limpias y sean comparables
df6_clean['Customer_ID'] = df6_clean['Customer_ID'].astype(str)
df7_imputado['Customer_ID'] = df7_imputado['Customer_ID'].astype(str)

# Columnas numéricas a imputar
cols_numericas_invalidas = [col for col in criterios_validacion.keys()]

# Iteramos sobre las filas con valores inválidos
for idx, row in registros_invalidos.iterrows():
    customer_id = row['Customer_ID']
    
    # Subset del cliente en el dataframe de entrenamiento
    historial_cliente = df6_clean[df6_clean['Customer_ID'] == customer_id]

    if historial_cliente.empty:
        continue  # No hay historial, se deja NaN

    for col in cols_numericas_invalidas:
        valor = row[col]
        if pd.isna(valor) or not criterios_validacion[col](valor):
            # Intentamos imputar con la moda válida del entrenamiento
            valores_validos = historial_cliente[col].dropna()
            valores_validos = valores_validos[valores_validos.apply(criterios_validacion[col])]

            if not valores_validos.empty:
                moda = valores_validos.mode().iloc[0]
                df7_imputado.at[idx, col] = moda  # Imputar en la copia limpia

In [12]:
# Repetimos el diagnóstico con la versión imputada
df_test_diagnostico_post = df7_imputado.copy()

# Reaplicamos la lógica de validación
for col in criterios_validacion:
    df_test_diagnostico_post[col] = pd.to_numeric(df_test_diagnostico_post[col], errors='coerce')
    df_test_diagnostico_post[f'{col}_valido'] = df_test_diagnostico_post[col].apply(
        lambda x: criterios_validacion[col](x) if not pd.isna(x) else False
    )

# Revisión final
df_test_diagnostico_post['Fila_Valida'] = df_test_diagnostico_post[
    [f'{col}_valido' for col in criterios_validacion]
].all(axis=1)

print(f"Registros aún inválidos tras imputación: {(~df_test_diagnostico_post['Fila_Valida']).sum()}")

Registros aún inválidos tras imputación: 23


In [13]:
# Definir columnas numéricas relevantes
cols_numericas = [
    'Age', 'Annual_Income', 'Monthly_Inhand_Salary', 'Num_Bank_Accounts', 
    'Num_Credit_Card', 'Interest_Rate', 'Num_of_Loan', 'Delay_from_due_date', 
    'Num_of_Delayed_Payment', 'Changed_Credit_Limit', 'Num_Credit_Inquiries', 
    'Outstanding_Debt', 'Credit_Utilization_Ratio', 'Credit_History_Age', 
    'Total_EMI_per_month', 'Amount_invested_monthly', 'Monthly_Balance'
]

# Identificar los Customer_ID con NaNs en columnas numéricas del test
nuevos_ids = df_test_diagnostico_post.loc[df_test_diagnostico_post[cols_numericas].isna().any(axis=1), 'Customer_ID'].unique()

# Customer_IDs presentes en el dataframe de entrenamiento
ids_en_train = df6_clean['Customer_ID'].unique()

# Filtrar los que realmente son nuevos
nuevos_ids_finales = [id_ for id_ in nuevos_ids if id_ not in ids_en_train]

print("Clientes realmente nuevos:", nuevos_ids_finales)
print("Total nuevos:", len(nuevos_ids_finales))
print('\nDado que los registros restantes con valores incoherentes no son clientes nuevos, se decide eliminarlos directamente')

# Eliminar registros donde la fila es inválida según la validación
df_test_diagnostico_post = df_test_diagnostico_post[df_test_diagnostico_post['Fila_Valida']]

# Verificación
print(f"[INFO] Nuevo tamaño del test set: {df_test_diagnostico_post.shape}")

# Volver a df7_clean
df7_clean = df_test_diagnostico_post.copy()

# Etiquetar como 'Unknown' valores NaN de la columna Name
df7_clean['Name'] = df7_clean['Name'].fillna('Unknown')

# Diagnosticamos nulos nuevamente
print('\nDiagnóstico de valores nulos luego de la limpieza')
for i in df7_clean.columns:
    print(i+':'+ str(df7_clean[i].isnull().sum()))

print('\nNuevamente podemos apreciar que la columna "Type_of_Loan" genera nulos, por lo tanto es necesario separar sus atributos multivaluados en columnas ' \
'individuales. Con ello se logra un mayor control en el preprocesamiento de este descriptor.')

Clientes realmente nuevos: []
Total nuevos: 0

Dado que los registros restantes con valores incoherentes no son clientes nuevos, se decide eliminarlos directamente
[INFO] Nuevo tamaño del test set: (46177, 45)

Diagnóstico de valores nulos luego de la limpieza
ID:0
Customer_ID:0
Month:0
Name:0
Age:0
SSN:0
Occupation:0
Annual_Income:0
Monthly_Inhand_Salary:0
Num_Bank_Accounts:0
Num_Credit_Card:0
Interest_Rate:0
Num_of_Loan:0
Type_of_Loan:3695
Delay_from_due_date:0
Num_of_Delayed_Payment:0
Changed_Credit_Limit:0
Num_Credit_Inquiries:0
Credit_Mix:0
Outstanding_Debt:0
Credit_Utilization_Ratio:0
Credit_History_Age:0
Payment_of_Min_Amount:0
Total_EMI_per_month:0
Amount_invested_monthly:0
Payment_Behaviour:0
Monthly_Balance:0
Age_valido:0
Annual_Income_valido:0
Monthly_Inhand_Salary_valido:0
Num_Bank_Accounts_valido:0
Num_Credit_Card_valido:0
Interest_Rate_valido:0
Num_of_Loan_valido:0
Delay_from_due_date_valido:0
Num_of_Delayed_Payment_valido:0
Changed_Credit_Limit_valido:0
Num_Credit_Inquir

In [14]:
# Separar columna 'Type_of_Loan' en categorías individuales para mejorar codificación y granularidad del dataframe
import pandas as pd
import numpy as np

# Crear una copia temporal del dataframe
df7_temp = df7_clean.copy()

# Limpiar comillas dobles, espacios y reemplazar "and" por ","
df7_temp['Type_of_Loan'] = (
    df7_temp['Type_of_Loan']
    .str.replace('"','',regex=False)
    .str.replace(' and ',', ', regex=False)
    .str.strip()
)

# Extraer todos los tipos únicos de prestamos (ignorando los NaN)
tipos_prestamo = set()
df7_temp['Type_of_Loan'].dropna().apply(lambda x: tipos_prestamo.update([i.strip() for i in x.split(',') if i.strip() != '']))

# Crear una columna por cada tipo de préstamo
for tipo in tipos_prestamo:
    df7_temp[tipo] = df7_temp['Type_of_Loan'].apply(
        lambda x: 'Yes' if pd.notna(x) and tipo in x else ('No' if pd.notna(x) else np.nan)
    )

# Eliminar la columna original
df7_temp.drop(columns=['Type_of_Loan'], inplace=True)

print(df7_temp)

# Diagnóstico
print(f"Tipos de préstamo extraídos: {sorted(tipos_prestamo)}")
print("Valores únicos por nueva columna (debería ser 'Yes', 'No' o NaN):\n")
print(df7_temp[list(tipos_prestamo)].nunique())

# Reasignamos al dataframe original

df7_clean = df7_temp.copy()

# Diagnosticamos nulos luego de la transformación de Type_of_Loan
print('\nDiagnóstico de valores nulos luego de la transformación de Type_of_Loan')
for i in df7_clean.columns:
    print(i+':'+ str(df7_clean[i].isnull().sum()))

print('\nNuevamente tenemos valores nulos luego de hacer el split en Type_of_Loan. Sin embargo, esta vez es podemos apoyarnos nuevamente en la información cruzada ' \
'de nuestro dataframe de entrenamiento para imputar información real (esta vez para una variable categórica). Este sería el último paso antes de tener que ' \
'rellenar con "Unknown". Por lo tanto, se llevará a cabo nuevamente la imputación mediante el dataframe train sin codificar (df6_clean)')

            ID Customer_ID      Month            Name   Age          SSN  \
0      0x16295  CUS_0x1000   December  Alistair Barrf  18.0  913-74-1218   
1      0x16294  CUS_0x1000   November  Alistair Barrf  18.0  913-74-1218   
2      0x16293  CUS_0x1000    October  Alistair Barrf  18.0  913-74-1218   
3      0x16292  CUS_0x1000  September  Alistair Barrf  18.0  913-74-1218   
4       0x66ad  CUS_0x1009   December          Arunah  26.0  063-67-6938   
...        ...         ...        ...             ...   ...          ...   
46195   0x61ee   CUS_0xffc  September           Brads  18.0  226-86-7294   
46196  0x25b05   CUS_0xffd   December        Damouniq  30.0  832-88-8320   
46197  0x25b04   CUS_0xffd   November        Damouniq  30.0  832-88-8320   
46198  0x25b03   CUS_0xffd    October        Damouniq  30.0  832-88-8320   
46199  0x25b02   CUS_0xffd  September        Damouniq  30.0  832-88-8320   

      Occupation  Annual_Income  Monthly_Inhand_Salary  Num_Bank_Accounts  \
0         

In [15]:
# Copia de trabajo para mantener df7_clean intacto por ahora
df7_temp = df7_clean.copy()

# Definimos las columnas individuales de préstamo
loan_cols = [
    'Auto Loan', 'Credit-Builder Loan', 'Debt Consolidation Loan', 'Home Equity Loan',
    'Mortgage Loan', 'Not Specified', 'Payday Loan', 'Personal Loan', 'Student Loan'
]

# Creamos un diccionario para guardar la moda por Customer_ID en el train
moda_prestamos_train = (
    df6_clean[['Customer_ID'] + loan_cols]
    .groupby('Customer_ID')
    .agg(lambda x: x.mode().iloc[0] if not x.mode().empty else np.nan)
)

# Imputamos en df7_temp
for col in loan_cols:
    nulos_antes = df7_temp[col].isna().sum()
    df7_temp[col] = df7_temp.apply(
        lambda row: moda_prestamos_train.loc[row['Customer_ID'], col]
        if pd.isna(row[col]) and row['Customer_ID'] in moda_prestamos_train.index
        else row[col],
        axis=1
    )
    print(f"[INFO] Imputados {nulos_antes - df7_temp[col].isna().sum()} valores en '{col}' usando historial del train set")

# Asignamos al df limpio
df7_clean = df7_temp.copy()

# Diagnóstico final
print("\nDiagnóstico de valores nulos tras imputación por Customer_ID:")
for col in loan_cols:
    print(f"{col}: {df7_clean[col].isna().sum()}")

print('\n Finalmente, se lograron imputar todos los valores nulos desde el dataframe de entrenamiento.')

#Diagnosticamos por última vez los valores nulos en las columnas de df7_clean
for i in df7_clean.columns:
    print(i+':'+ str(df7_clean[i].isnull().sum()))

[INFO] Imputados 3695 valores en 'Auto Loan' usando historial del train set
[INFO] Imputados 3695 valores en 'Credit-Builder Loan' usando historial del train set
[INFO] Imputados 3695 valores en 'Debt Consolidation Loan' usando historial del train set
[INFO] Imputados 3695 valores en 'Home Equity Loan' usando historial del train set
[INFO] Imputados 3695 valores en 'Mortgage Loan' usando historial del train set
[INFO] Imputados 3695 valores en 'Not Specified' usando historial del train set
[INFO] Imputados 3695 valores en 'Payday Loan' usando historial del train set
[INFO] Imputados 3695 valores en 'Personal Loan' usando historial del train set
[INFO] Imputados 3695 valores en 'Student Loan' usando historial del train set

Diagnóstico de valores nulos tras imputación por Customer_ID:
Auto Loan: 0
Credit-Builder Loan: 0
Debt Consolidation Loan: 0
Home Equity Loan: 0
Mortgage Loan: 0
Not Specified: 0
Payday Loan: 0
Personal Loan: 0
Student Loan: 0

 Finalmente, se lograron imputar todos 

In [16]:
# Verificar detalles finales de limpieza 

print("\n[INFO] Ocupaciones únicas antes de limpiar:")
print(sorted(df7_clean['Occupation'].dropna().unique()))

print('\n[INFO] Observamos que existen registros con el valor sucio "_______". Se inicia proceso de imputación por duplicado...')

# Creamos una copia para no afectar el original
df7_temp = df7_clean.copy()

# Creamos la máscara inicial
mascara_sucia = df7_temp['Occupation'] == '_______'
sucios_previos = mascara_sucia.sum()

# Bucle con bandera de control
while True:
    # Calcular la moda por Customer_ID (excluyendo registros sucios)
    moda_occupation_por_cliente = (
        df7_temp[~mascara_sucia]
        .groupby('Customer_ID')['Occupation']
        .agg(lambda x: x.mode().iloc[0] if not x.mode().empty else np.nan)
    )

    # Imputar donde sea posible
    for idx in df7_temp[mascara_sucia].index:
        customer = df7_temp.at[idx, 'Customer_ID']
        ocupacion_moda = moda_occupation_por_cliente.get(customer, np.nan)
        if pd.notna(ocupacion_moda):
            df7_temp.at[idx, 'Occupation'] = ocupacion_moda

    # Recalcular la máscara y comparar
    mascara_sucia = df7_temp['Occupation'] == '_______'
    sucios_actuales = mascara_sucia.sum()
    print(f"[DEBUG] Registros aún sucios: {sucios_actuales}")

    # Verificar si no hubo progreso → cortar
    if sucios_actuales == sucios_previos:
        print("[INFO] No se puede imputar más. Se procede a reemplazar por 'Unknown'.")
        break

    sucios_previos = sucios_actuales

# Reemplazar los que no pudieron imputarse
df7_temp.loc[df7_temp['Occupation'] == '_______', 'Occupation'] = 'Unknown'

# Asignar al dataframe limpio final
df7_clean = df7_temp.copy()

# Verificación final
print("\n[INFO] Ocupaciones únicas después de limpieza:")
print(sorted(df7_clean['Occupation'].dropna().unique()))


[INFO] Ocupaciones únicas antes de limpiar:
['Accountant', 'Architect', 'Developer', 'Doctor', 'Engineer', 'Entrepreneur', 'Journalist', 'Lawyer', 'Manager', 'Mechanic', 'Media_Manager', 'Musician', 'Scientist', 'Teacher', 'Writer', '_______']

[INFO] Observamos que existen registros con el valor sucio "_______". Se inicia proceso de imputación por duplicado...
[DEBUG] Registros aún sucios: 12
[DEBUG] Registros aún sucios: 12
[INFO] No se puede imputar más. Se procede a reemplazar por 'Unknown'.

[INFO] Ocupaciones únicas después de limpieza:
['Accountant', 'Architect', 'Developer', 'Doctor', 'Engineer', 'Entrepreneur', 'Journalist', 'Lawyer', 'Manager', 'Mechanic', 'Media_Manager', 'Musician', 'Scientist', 'Teacher', 'Unknown', 'Writer']


In [17]:
#Limpiar columnas sin información real interpolable

print(sorted(df7_clean['Credit_Mix'].dropna().unique()))

# Etiquetar como 'Unknown' valores vacíos de la columna Credit_Mix
df7_clean['Credit_Mix'] = df7_clean['Credit_Mix'].replace(' ','Unknown')

#Diagnosticamos por última vez los valores nulos en las columnas del dataframe
for i in df7_clean.columns:
    print(i+':'+ str(df7_clean[i].isnull().sum()))

print(sorted(df7_clean['Credit_Mix'].dropna().unique()))

[' ', 'Bad', 'Good', 'Standard']
ID:0
Customer_ID:0
Month:0
Name:0
Age:0
SSN:0
Occupation:0
Annual_Income:0
Monthly_Inhand_Salary:0
Num_Bank_Accounts:0
Num_Credit_Card:0
Interest_Rate:0
Num_of_Loan:0
Delay_from_due_date:0
Num_of_Delayed_Payment:0
Changed_Credit_Limit:0
Num_Credit_Inquiries:0
Credit_Mix:0
Outstanding_Debt:0
Credit_Utilization_Ratio:0
Credit_History_Age:0
Payment_of_Min_Amount:0
Total_EMI_per_month:0
Amount_invested_monthly:0
Payment_Behaviour:0
Monthly_Balance:0
Age_valido:0
Annual_Income_valido:0
Monthly_Inhand_Salary_valido:0
Num_Bank_Accounts_valido:0
Num_Credit_Card_valido:0
Interest_Rate_valido:0
Num_of_Loan_valido:0
Delay_from_due_date_valido:0
Num_of_Delayed_Payment_valido:0
Changed_Credit_Limit_valido:0
Num_Credit_Inquiries_valido:0
Outstanding_Debt_valido:0
Credit_Utilization_Ratio_valido:0
Credit_History_Age_valido:0
Total_EMI_per_month_valido:0
Amount_invested_monthly_valido:0
Monthly_Balance_valido:0
Fila_Valida:0
Payday Loan:0
Not Specified:0
Mortgage Loan:

In [18]:
# Eliminar columnas de validez que ya no son útiles
cols_validez = [col for col in df7_clean.columns if col.endswith('_valido') or col == 'Fila_Valida']
df7_clean.drop(columns=cols_validez, inplace=True)

# Verificación
print(f"[INFO] Columnas eliminadas: {cols_validez}")
print(f"[INFO] Tamaño final del dataframe: {df7_clean.shape}")

[INFO] Columnas eliminadas: ['Age_valido', 'Annual_Income_valido', 'Monthly_Inhand_Salary_valido', 'Num_Bank_Accounts_valido', 'Num_Credit_Card_valido', 'Interest_Rate_valido', 'Num_of_Loan_valido', 'Delay_from_due_date_valido', 'Num_of_Delayed_Payment_valido', 'Changed_Credit_Limit_valido', 'Num_Credit_Inquiries_valido', 'Outstanding_Debt_valido', 'Credit_Utilization_Ratio_valido', 'Credit_History_Age_valido', 'Total_EMI_per_month_valido', 'Amount_invested_monthly_valido', 'Monthly_Balance_valido', 'Fila_Valida']
[INFO] Tamaño final del dataframe: (46177, 35)


In [19]:
df7_clean.to_csv('test_clean(sin_codificar).csv', index=False)

import os
print(os.getcwd())

d:\Ciencia de Datos y Análisis con Python\Bases de Datos Kaggle\Credit Score Classification feb\Archivos Repositorio Github


In [20]:
print(df7_clean.dtypes)

ID                           object
Customer_ID                  object
Month                        object
Name                         object
Age                         float64
SSN                          object
Occupation                   object
Annual_Income               float64
Monthly_Inhand_Salary       float64
Num_Bank_Accounts             int64
Num_Credit_Card               int64
Interest_Rate                 int64
Num_of_Loan                 float64
Delay_from_due_date           int64
Num_of_Delayed_Payment      float64
Changed_Credit_Limit        float64
Num_Credit_Inquiries        float64
Credit_Mix                   object
Outstanding_Debt            float64
Credit_Utilization_Ratio    float64
Credit_History_Age          float64
Payment_of_Min_Amount        object
Total_EMI_per_month         float64
Amount_invested_monthly     float64
Payment_Behaviour            object
Monthly_Balance             float64
Payday Loan                  object
Not Specified               

In [21]:
#Codificación de variables categóricas dataframe de entrenamiento

import pandas as pd
from sklearn.preprocessing import LabelEncoder, OneHotEncoder

# Copia del dataframe limpio
df7_cleaned_final = df7_clean.copy()

# Limpieza de 'Occupation'

# Lista completa de ocupaciones válidas
ocupaciones_validas = [
    'Lawyer', 'Mechanic', 'Media_Manager', 'Doctor', 'Journalist',
    'Accountant', 'Manager', 'Entrepreneur', 'Scientist', 'Architect',
    'Teacher', 'Engineer', 'Writer', 'Developer', 'Musician', 'Unknown'
]

# Reemplazar cualquier valor no listado por 'Other'
df7_cleaned_final['Occupation'] = df7_cleaned_final['Occupation'].apply(
    lambda x: x if x in ocupaciones_validas else 'Other'
)

# Definimos un mapeo numérico del 1 al 17 (sin significado ordinal) para los valores de Occupation
occupation_categories = ocupaciones_validas + ['Other']
occupation_mapping = {name: idx + 1 for idx, name in enumerate(occupation_categories)}
df7_cleaned_final['Occupation_Encoded'] = df7_cleaned_final['Occupation'].map(occupation_mapping)

#Verificación
print('Verificación codificación Occupation')
print(df7_cleaned_final[['Occupation', 'Occupation_Encoded']].drop_duplicates().sort_values('Occupation_Encoded'))

# Reemplazar columna original por codificada (Occupation_Encoded)
df7_cleaned_final['Occupation'] = df7_cleaned_final['Occupation_Encoded']
df7_cleaned_final.drop(columns=['Occupation'], inplace=True)

# Codificación de ordinales

# Credit_Mix (ordinal: Bad < Standard < Good, Unknown)
mix_mapping = {'Bad': 0, 'Standard': 1, 'Good': 2, 'Unknown':3}
df7_cleaned_final['Credit_Mix_Encoded'] = df7_cleaned_final['Credit_Mix'].map(mix_mapping)

# Payment_of_Min_Amount (ordinal: NM < No < Yes)
paymin_mapping = {'NM': 0, 'No': 1, 'Yes': 2}
df7_cleaned_final['Payment_of_Min_Amount_Encoded'] = df7_cleaned_final['Payment_of_Min_Amount'].map(paymin_mapping)

# Eliminamos las columnas originales de Credit_Mix y Payment_of_Min_Amount 
df7_cleaned_final.drop(columns=['Credit_Mix', 'Payment_of_Min_Amount'], inplace=True)

# Definir mapeo de categorías de Payment_Behaviour
payment_behaviour_mapping = {
    'High spent Medium value payments': 1,
    'High spent Small value payments': 2,
    'Low spent Large value payments': 3,
    'Low spent Small value payments': 4,
    'Low spent Medium value payments': 5,
    'High spent Large value payments': 6
}

# Asignar valor 7 a cualquier valor que no esté en el diccionario (Other)
df7_cleaned_final['Payment_Behaviour_Encoded'] = df7_cleaned_final['Payment_Behaviour'].map(payment_behaviour_mapping).fillna(7)

# Reemplazar columna original
df7_cleaned_final['Payment_Behaviour'] = df7_cleaned_final['Payment_Behaviour_Encoded']
df7_cleaned_final.drop(columns=['Payment_Behaviour_Encoded'], inplace=True)

# Codificación de columnas individuales de préstamos

loan_cols = [
    'Auto Loan', 'Credit-Builder Loan', 'Debt Consolidation Loan', 'Home Equity Loan',
    'Mortgage Loan', 'Not Specified', 'Payday Loan', 'Personal Loan', 'Student Loan'
]

# Mapeo Yes/No/Unknown → 2/1/0
loan_mapping = {'Unknown': 0, 'No': 1, 'Yes': 2}

for col in loan_cols:
    df7_cleaned_final[col + ' Encoded'] = df7_cleaned_final[col].map(loan_mapping)

# Eliminar columnas de texto de préstamos
df7_cleaned_final.drop(columns=loan_cols, inplace=True)

# Verificación final

print("Dimensiones finales del dataset codificado:", df7_cleaned_final.shape)
print("Columnas codificadas (últimas):", df7_cleaned_final.columns[-10:].tolist())
print(df7_cleaned_final)
print(df7_cleaned_final['Credit_Mix_Encoded'])
print(df7_cleaned_final['Payment_of_Min_Amount_Encoded'])

Verificación codificación Occupation
         Occupation  Occupation_Encoded
0            Lawyer                   1
4          Mechanic                   2
8     Media_Manager                   3
11           Doctor                   4
18       Journalist                   5
22       Accountant                   6
26          Manager                   7
30     Entrepreneur                   8
23        Scientist                   9
45        Architect                  10
52          Teacher                  11
75         Engineer                  12
83           Writer                  13
102       Developer                  14
129        Musician                  15
5731        Unknown                  16
Dimensiones finales del dataset codificado: (46177, 35)
Columnas codificadas (últimas): ['Payment_of_Min_Amount_Encoded', 'Auto Loan Encoded', 'Credit-Builder Loan Encoded', 'Debt Consolidation Loan Encoded', 'Home Equity Loan Encoded', 'Mortgage Loan Encoded', 'Not Specified Encode

In [22]:
# Estandarización de variables numéricas

from sklearn.preprocessing import MinMaxScaler

# Columnas numéricas relevantes para similitud
cols_numericas = [
    'Age', 'Annual_Income', 'Monthly_Inhand_Salary', 'Num_Bank_Accounts', 
    'Num_Credit_Card', 'Interest_Rate', 'Num_of_Loan', 'Delay_from_due_date', 
    'Num_of_Delayed_Payment', 'Changed_Credit_Limit', 'Num_Credit_Inquiries', 
    'Outstanding_Debt', 'Credit_Utilization_Ratio', 'Credit_History_Age', 
    'Total_EMI_per_month', 'Amount_invested_monthly', 'Monthly_Balance'
]

scaler = MinMaxScaler()
df7_cleaned_final[cols_numericas] = scaler.fit_transform(df7_cleaned_final[cols_numericas])

columnas_a_imprimir = [
    'Age', 'Annual_Income', 'Monthly_Inhand_Salary', 'Num_Bank_Accounts', 
    'Num_Credit_Card', 'Interest_Rate', 'Num_of_Loan', 'Delay_from_due_date', 
    'Num_of_Delayed_Payment', 'Changed_Credit_Limit', 'Num_Credit_Inquiries', 
    'Outstanding_Debt', 'Credit_Utilization_Ratio', 'Credit_History_Age', 
    'Total_EMI_per_month', 'Amount_invested_monthly', 'Monthly_Balance'
]

#Verificación estandarización columnas numéricas
print(df7_cleaned_final[columnas_a_imprimir])

            Age  Annual_Income  Monthly_Inhand_Salary  Num_Bank_Accounts  \
0      0.095238       0.004538               0.161232           0.272727   
1      0.095238       0.004538               0.161232           0.272727   
2      0.095238       0.004538               0.161232           0.272727   
3      0.095238       0.004538               0.161232           0.272727   
4      0.285714       0.000089               0.046034           0.545455   
...         ...            ...                    ...                ...   
46195  0.095238       0.002566               0.329807           0.545455   
46196  0.380952       0.001638               0.231275           0.727273   
46197  0.380952       0.001638               0.231275           0.727273   
46198  0.380952       0.001638               0.231275           0.727273   
46199  0.380952       0.001638               0.231275           0.727273   

       Num_Credit_Card  Interest_Rate  Num_of_Loan  Delay_from_due_date  \
0           

In [23]:
#Diagnosticamos por última vez los valores nulos en las columnas del dataframe
for i in df7_cleaned_final.columns:
    print(i+':'+ str(df7_cleaned_final[i].isnull().sum()))

ID:0
Customer_ID:0
Month:0
Name:0
Age:0
SSN:0
Annual_Income:0
Monthly_Inhand_Salary:0
Num_Bank_Accounts:0
Num_Credit_Card:0
Interest_Rate:0
Num_of_Loan:0
Delay_from_due_date:0
Num_of_Delayed_Payment:0
Changed_Credit_Limit:0
Num_Credit_Inquiries:0
Outstanding_Debt:0
Credit_Utilization_Ratio:0
Credit_History_Age:0
Total_EMI_per_month:0
Amount_invested_monthly:0
Payment_Behaviour:0
Monthly_Balance:0
Occupation_Encoded:0
Credit_Mix_Encoded:0
Payment_of_Min_Amount_Encoded:0
Auto Loan Encoded:0
Credit-Builder Loan Encoded:0
Debt Consolidation Loan Encoded:0
Home Equity Loan Encoded:0
Mortgage Loan Encoded:0
Not Specified Encoded:0
Payday Loan Encoded:0
Personal Loan Encoded:0
Student Loan Encoded:0


In [24]:
# Detectar errores pre-exportación del csv

import re

# Copia de seguridad antes de limpiar
df_temp = df7_cleaned_final.copy()

# Patrón de caracteres problemáticos
patron_caracteres_invalidos = r'["\',]'

# Inicializar bandera
nombres_previos_invalidos = -1

while True:
    # Detectar nombres con caracteres indeseados
    errores_nombres = df_temp[df_temp['Name'].str.contains(patron_caracteres_invalidos, regex=True, na=False)]

    # Ver cuántos quedan
    errores_actuales = len(errores_nombres)

    print(f"[DEBUG] Nombres con caracteres problemáticos: {errores_actuales}")

    # Si no hay avance (mismo número de errores), salimos del bucle
    if errores_actuales == nombres_previos_invalidos:
        print("[INFO] No se detecta avance, se asume limpieza finalizada.")
        break

    # Actualizar bandera
    nombres_previos_invalidos = errores_actuales

    # Corrección de errores
    df_temp['Name'] = (
        df_temp['Name']
        .str.replace('"', '', regex=False)
        .str.replace("'", '', regex=False)
        .str.replace(r',+', ',', regex=True)  # múltiples comas → una sola
        .str.replace(',', '', regex=False)    # luego eliminar las comas
        .str.replace(' ', '', regex=False)    # eliminar espacios
        .str.strip()                          # limpiar bordes
    )

# Asignamos a df limpio final
df7_cleaned_final = df_temp.copy()

# Verificación final de los primeros valores únicos
print("\n[INFO] Primeros valores únicos de la columna Name post-limpieza:")
print(df7_cleaned_final['Name'].dropna().unique()[:30])

[DEBUG] Nombres con caracteres problemáticos: 446
[DEBUG] Nombres con caracteres problemáticos: 0
[DEBUG] Nombres con caracteres problemáticos: 0
[INFO] No se detecta avance, se asume limpieza finalizada.

[INFO] Primeros valores únicos de la columna Name post-limpieza:
['AlistairBarrf' 'Arunah' 'Shirboni' 'Schneyerh' 'Cameront' 'Holtono'
 'Felsenthalq' 'Josephv' 'NeilChatterjeex' 'Rhysn' 'Wahbap' 'Matthewm'
 'B.h' 'TimKellyf' 'JonathanStempela' 'MakiShirakip' 'JanetMcGurtyg'
 'Leahk' 'Kentaros' 'Markm' 'LuciaMutikanin' 'Hornbyb' 'DavidSheppardv'
 'PatriciaZengerlel' 'Lopezz' 'SylviaWestallq' 'AndreasCremero'
 'AlwynScottk' 'Dorfmanv' 'Karenh']


In [25]:
df7_cleaned_final.to_csv('test_clean(codificado).csv', index=False)

import os
print(os.getcwd())

d:\Ciencia de Datos y Análisis con Python\Bases de Datos Kaggle\Credit Score Classification feb\Archivos Repositorio Github


In [26]:
print(df7_cleaned_final.dtypes)

ID                                  object
Customer_ID                         object
Month                               object
Name                                object
Age                                float64
SSN                                 object
Annual_Income                      float64
Monthly_Inhand_Salary              float64
Num_Bank_Accounts                  float64
Num_Credit_Card                    float64
Interest_Rate                      float64
Num_of_Loan                        float64
Delay_from_due_date                float64
Num_of_Delayed_Payment             float64
Changed_Credit_Limit               float64
Num_Credit_Inquiries               float64
Outstanding_Debt                   float64
Credit_Utilization_Ratio           float64
Credit_History_Age                 float64
Total_EMI_per_month                float64
Amount_invested_monthly            float64
Payment_Behaviour                    int64
Monthly_Balance                    float64
Occupation_